In [ ]:
print("Hello World")

## 공통 모듈 불러와서 사용하기

In [36]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))
from common.hello import *
from common.yfinance import *

hello()

Hello MS AI School Project Team 6


## 필요한 라이브러리 import, 그래프 출력 한글 처리

In [37]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import gradio as gr
from io import BytesIO
from PIL import Image

import platform
from matplotlib import font_manager, rc

def set_korean_font():
    if platform.system() == 'Windows':
        # Windows의 경우
        font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
        rc('font', family=font_name)
    elif platform.system() == 'Darwin':
        # macOS의 경우
        rc('font', family='AppleGothic')
    else:
        # 기타 OS의 경우
        rc('font', family='NanumGothic')

# 한글 폰트 설정
set_korean_font()

## 주식 데이터를 yfinance 에서 가져와서 데이터프레임과 그래프로 출력하는 함수

In [ ]:
''' 공통 모듈로 분리
# 주식 데이터를 가져오고 차트를 생성하는 함수
def get_stock_data(ticker):
    # yfinance를 이용해 주식 데이터 가져오기 (최근 1년 데이터)
    data = yf.download(ticker, period="1y")
    
    # 결과가 없으면 안내 문구 반환
    if data.empty:
        return "데이터가 없습니다. 유효한 티커를 입력하세요.", None
    
    # 데이터프레임을 정리하여 표시할 정보 선택
    data_display = data[['Open', 'High', 'Low', 'Close', 'Volume']]
    
    # 열 이름을 문자열로 설정하여 문제 해결
    data_display.columns = [col[0] if isinstance(col, tuple) else col for col in data_display.columns]
    
    # 인덱스를 열로 변환
    data_display.reset_index(inplace=True)
    
    # 날짜 정보를 년월일 정보만 출력
    data_display['Date'] = data_display['Date'].dt.strftime('%Y-%m-%d')
    
    # 나머지 값들은 소수점 둘째자리까지 출력하고 천단위 컴마(,)를 붙여줌
    # data_display = data_display.round(2)
    data_display['Open'] = data_display['Open'].apply(lambda x: f"{x:,.2f}")
    data_display['High'] = data_display['High'].apply(lambda x: f"{x:,.2f}")
    data_display['Low'] = data_display['Low'].apply(lambda x: f"{x:,.2f}")
    data_display['Close'] = data_display['Close'].apply(lambda x: f"{x:,.2f}")
    data_display['Volume'] = data_display['Volume'].apply(lambda x: f"{x:,}")
    
    
    # 차트 생성
    plt.figure(figsize=(10, 5))
    plt.plot(data['Close'], label="Close Price", color='blue')
    plt.title(f"{ticker} - 일별 종가 차트")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend()
    plt.grid()
    
    # 이미지를 메모리에 저장
    buf = BytesIO()
    plt.savefig(buf, format="png")
    plt.close()
    buf.seek(0)  # 버퍼의 시작으로 이동
    img = Image.open(buf)  # PIL 이미지로 변환
    
    data_display = data_display.iloc[::-1].reset_index(drop=True)
    return data_display, img
'''

## Gradio UI 구성

In [ ]:
# Gradio 인터페이스 설정
with gr.Blocks(gr.themes.Monochrome()) as demo:
    gr.Markdown("# 주식 데이터 조회")
    ticker_input = gr.Textbox(label="주식 티커 입력", placeholder="예: AAPL, MSFT 등")
    with gr.Row():
        submit_btn = gr.Button("조회")
        reset_btn = gr.Button("초기화")
    data_output = gr.Dataframe(label="일별 주가 데이터", visible=False) 
    chart_output = gr.Image(label="주가 차트", visible=False)
    
    def update_visibility(ticker):
        ticker = ticker.upper()  # Convert ticker to uppercase
        data, img = get_yfinance_stock_data(ticker)
        return gr.update(visible=True), gr.update(visible=True), data, img
    
    def reset_fields():
        return "", gr.update(visible=False), gr.update(visible=False)
    
    submit_btn.click(update_visibility, inputs=ticker_input, outputs=[data_output, chart_output, data_output, chart_output])
    ticker_input.submit(update_visibility, inputs=ticker_input, outputs=[data_output, chart_output, data_output, chart_output])
    reset_btn.click(reset_fields, inputs=None, outputs=[ticker_input, data_output, chart_output])

# Gradio 앱 실행
demo.launch()
# demo.launch(share=True)

## 조회 기간 설정하여 조회되도록 수정

In [ ]:
''' 공통 모듈로 분리
# 주식 데이터를 가져오고 차트를 생성하는 함수
def get_stock_data(ticker, start, end):
    # yfinance를 이용해 주식 데이터 가져오기 (지정된 기간 데이터)
    data = yf.download(ticker, start=start, end=end)
    
    # 결과가 없으면 안내 문구 반환
    if data.empty:
        return "데이터가 없습니다. 유효한 티커를 입력하세요.", None
    
    # 데이터프레임을 정리하여 표시할 정보 선택
    data_display = data[['Open', 'High', 'Low', 'Close', 'Volume']]
    
    # 열 이름을 문자열로 설정하여 문제 해결
    data_display.columns = [col[0] if isinstance(col, tuple) else col for col in data_display.columns]
    
    # 인덱스를 열로 변환
    data_display.reset_index(inplace=True)
    
    # 날짜 정보를 년월일 정보만 출력
    data_display['Date'] = data_display['Date'].dt.strftime('%Y-%m-%d')
    
    # 나머지 값들은 소수점 둘째자리까지 출력하고 천단위 컴마(,)를 붙여줌
    data_display['Open'] = data_display['Open'].apply(lambda x: f"{x:,.2f}")
    data_display['High'] = data_display['High'].apply(lambda x: f"{x:,.2f}")
    data_display['Low'] = data_display['Low'].apply(lambda x: f"{x:,.2f}")
    data_display['Close'] = data_display['Close'].apply(lambda x: f"{x:,.2f}")
    data_display['Volume'] = data_display['Volume'].apply(lambda x: f"{x:,}")
    
    # 차트 생성
    plt.figure(figsize=(10, 5))
    plt.plot(data['Close'], label="Close Price", color='blue')
    plt.title(f"{ticker} - 일별 종가 차트")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend()
    plt.grid()
    
    # 이미지를 메모리에 저장
    buf = BytesIO()
    plt.savefig(buf, format="png")
    plt.close()
    buf.seek(0)  # 버퍼의 시작으로 이동
    img = Image.open(buf)  # PIL 이미지로 변환
    
    data_display = data_display.iloc[::-1].reset_index(drop=True)
    return data_display, img
'''

## 시작일 종료일 지정하는 UI

In [ ]:
# Gradio 인터페이스 설정
with gr.Blocks(gr.themes.Monochrome()) as demo:
    gr.Markdown("# 주식 데이터 조회")
    ticker_input = gr.Textbox(label="주식 티커 입력", placeholder="예: AAPL, MSFT 등")
    start_date = gr.Textbox(label="조회 시작일", placeholder="예: 2022-01-01")
    end_date = gr.Textbox(label="조회 종료일", placeholder="예: 2023-01-01")
    with gr.Row():
        submit_btn = gr.Button("조회")
        reset_btn = gr.Button("초기화")
    data_output = gr.Dataframe(label="일별 주가 데이터", visible=False) 
    chart_output = gr.Image(label="주가 차트", visible=False)
    
    def update_visibility(ticker, start_date, end_date):
        ticker = ticker.upper()  # Convert ticker to uppercase
        data, img = get_yfinance_stock_data(ticker, start=start_date, end=end_date)
        return gr.update(visible=True), gr.update(visible=True), data, img
    
    def reset_fields():
        return "", "", "", gr.update(visible=False), gr.update(visible=False)
    
    submit_btn.click(update_visibility, inputs=[ticker_input, start_date, end_date], outputs=[data_output, chart_output, data_output, chart_output])
    ticker_input.submit(update_visibility, inputs=[ticker_input, start_date, end_date], outputs=[data_output, chart_output, data_output, chart_output])
    reset_btn.click(reset_fields, inputs=None, outputs=[ticker_input, start_date, end_date, data_output, chart_output])

# Gradio 앱 실행
demo.launch()
# demo.launch(share=True)

In [ ]:
start_date = "2022-01-01"
end_date = "2022-01-10"
get_yfinance_stock_data("AAPL", start=start_date, end=end_date)

## Gradio UI 연습, 2개의 종목을 한번에 조회

In [ ]:
'''
# Gradio 인터페이스 설정
with gr.Blocks(gr.themes.Monochrome()) as demo:
    gr.Markdown("# 주식 데이터 조회")
    with gr.Row():
        ticker_input1 = gr.Textbox(label="첫 번째 주식 티커 입력", placeholder="예: AAPL")
        ticker_input2 = gr.Textbox(label="두 번째 주식 티커 입력", placeholder="예: MSFT")
    with gr.Row():
        submit_btn = gr.Button("조회")
        reset_btn = gr.Button("초기화")
    with gr.Row():
        data_output1 = gr.Dataframe(label="첫 번째 종목 일별 주가 데이터", visible=False)
        data_output2 = gr.Dataframe(label="두 번째 종목 일별 주가 데이터", visible=False)
    with gr.Row():
        chart_output1 = gr.Image(label="첫 번째 종목 주가 차트", visible=False)
        chart_output2 = gr.Image(label="두 번째 종목 주가 차트", visible=False)
    
    def update_visibility(ticker1, ticker2):
        data1, img1 = get_stock_data(ticker1) if ticker1 else (None, None)
        data2, img2 = get_stock_data(ticker2) if ticker2 else (None, None)
        
        return (
            gr.update(visible=True) if data1 is not None else gr.update(visible=False),
            gr.update(visible=True) if data2 is not None else gr.update(visible=False),
            gr.update(visible=True) if img1 is not None else gr.update(visible=False),
            gr.update(visible=True) if img2 is not None else gr.update(visible=False),
            data1 if data1 is not None else gr.update(visible=False),
            data2 if data2 is not None else gr.update(visible=False),
            img1,
            img2
        )
    
    def reset_fields():
        return "", "", gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)
    
    submit_btn.click(update_visibility, inputs=[ticker_input1, ticker_input2], outputs=[data_output1, data_output2, chart_output1, chart_output2, data_output1, data_output2, chart_output1, chart_output2])
    ticker_input1.submit(update_visibility, inputs=[ticker_input1, ticker_input2], outputs=[data_output1, data_output2, chart_output1, chart_output2, data_output1, data_output2, chart_output1, chart_output2])
    ticker_input2.submit(update_visibility, inputs=[ticker_input1, ticker_input2], outputs=[data_output1, data_output2, chart_output1, chart_output2, data_output1, data_output2, chart_output1, chart_output2])
    reset_btn.click(reset_fields, inputs=None, outputs=[ticker_input1, ticker_input2, data_output1, data_output2, chart_output1, chart_output2])

# Gradio 앱 실행
demo.launch()
# demo.launch(share=True)
'''

## 이전 조회한 종목을 바로 선택할 수 있는 UI

In [ ]:
# Gradio 인터페이스 설정
with gr.Blocks(gr.themes.Monochrome()) as demo:
    previous_tickers = gr.State(set())
    gr.Markdown("# 주식 데이터 조회")
    previous_tickers_display = gr.Radio(label="이전 조회된 티커", choices=[], interactive=True, visible=False)
    ticker_input = gr.Textbox(label="주식 티커 입력", placeholder="예: AAPL, MSFT 등")
    warning_message = gr.Markdown(value="", visible=False)
    with gr.Row():
        submit_btn = gr.Button("조회")
        reset_btn = gr.Button("초기화")
    data_output = gr.Dataframe(label="일별 주가 데이터", visible=False, render=False) 
    chart_output = gr.Image(label="주가 차트", visible=False)
    
    def update_visibility(ticker, previous_tickers):
        if not ticker:
            return gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="티커를 입력하세요.", visible=True)
        
        ticker = ticker.upper()  # Convert ticker to uppercase
        data, img = get_yfinance_stock_data(ticker)
        previous_tickers.add(ticker)
        return (
            gr.update(visible=True, label=f"일별 주가 데이터 [{ticker}]"),
            gr.update(visible=True),
            data,
            img,
            gr.update(choices=list(previous_tickers), visible=True),
            gr.update(visible=False)
        )
    
    def reset_fields():
        return "", gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)
    
    def update_ticker_input(selected_ticker):
        return selected_ticker, gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(value="", visible=False)
    
    def auto_submit_ticker(selected_ticker, previous_tickers):
        return update_visibility(selected_ticker, previous_tickers)
    
    submit_btn.click(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, data_output, chart_output, previous_tickers_display, warning_message])
    ticker_input.submit(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, data_output, chart_output, previous_tickers_display, warning_message])
    reset_btn.click(reset_fields, inputs=None, outputs=[ticker_input, data_output, chart_output, warning_message])
    previous_tickers_display.change(update_ticker_input, inputs=previous_tickers_display, outputs=[ticker_input, data_output, chart_output, warning_message])
    previous_tickers_display.change(auto_submit_ticker, inputs=[previous_tickers_display, previous_tickers], outputs=[data_output, chart_output, data_output, chart_output, previous_tickers_display, warning_message])

# Gradio 앱 실행
demo.launch()
# demo.launch(share=True)


## 가격에측 가능한(모델 학습이 완료된) 티커에 대해서만 조회 되도록 수정

In [ ]:
import random

# 사용 가능한 티커 목록 가져오기
available_tickers = pd.read_csv('../data/available_tickers.csv')['tickers'].str.upper().tolist()
random.shuffle(available_tickers)

# Gradio 인터페이스 설정
with gr.Blocks(gr.themes.Monochrome()) as demo:
    previous_tickers = gr.State(set())
    gr.Markdown("# 주식 데이터 조회")
    previous_tickers_display = gr.Radio(label="이전 조회된 티커", choices=[], interactive=True, visible=False)
    ticker_input = gr.Textbox(label="주식 티커 입력", placeholder=f"예: {", ".join(available_tickers[:5])} 등")
    warning_message = gr.Markdown(value="", visible=False)
    with gr.Row():
        submit_btn = gr.Button("조회")
        reset_btn = gr.Button("초기화")

    chart_output = gr.Image(label="주가 차트", visible=False)
    data_output = gr.Dataframe(label="일별 주가 데이터", visible=False)
    
    def update_visibility(ticker, previous_tickers):
        if not ticker:
            return gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="티커를 입력하세요.", visible=True)
        
        ticker = ticker.upper()  # Convert ticker to uppercase
        if ticker.upper() not in available_tickers:
            return gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="입력하신 종목의 가격 예측은 추후에 제공될 예정입니다..", visible=True)
        
        data, img = get_yfinance_stock_data(ticker)
        previous_tickers.add(ticker)
        return (
            gr.update(visible=True, label=f"일별 주가 데이터 [{ticker}]"),
            gr.update(visible=True),
            data,
            img,
            gr.update(choices=list(previous_tickers), visible=True),
            gr.update(visible=False)
        )
    
    def reset_fields():
        return "", gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)
    
    def update_ticker_input(selected_ticker):
        return selected_ticker, gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(value="", visible=False)
    
    def auto_submit_ticker(selected_ticker, previous_tickers):
        return update_visibility(selected_ticker, previous_tickers)
    
    submit_btn.click(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, data_output, chart_output, previous_tickers_display, warning_message])
    ticker_input.submit(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, data_output, chart_output, previous_tickers_display, warning_message])
    reset_btn.click(reset_fields, inputs=None, outputs=[ticker_input, data_output, chart_output, warning_message])
    previous_tickers_display.change(update_ticker_input, inputs=previous_tickers_display, outputs=[ticker_input, data_output, chart_output, warning_message])
    previous_tickers_display.change(auto_submit_ticker, inputs=[previous_tickers_display, previous_tickers], outputs=[data_output, chart_output, data_output, chart_output, previous_tickers_display, warning_message])

# Gradio 앱 실행
demo.launch()
# demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7886

To create a public link, set `share=True` in `launch()`.


[*********************100%***********************]  1 of 1 completed


## 배경 이미지 추가

In [ ]:
import random

# 사용 가능한 티커 목록 가져오기
available_tickers = pd.read_csv('../data/available_tickers.csv')['tickers'].str.upper().tolist()
random.shuffle(available_tickers)

# Gradio 인터페이스 설정
css_code = f"""
.gradio-container {{
    background-image: url(data:image/png;base64,{encoded_string});
    background-size: cover;
    background-repeat: no-repeat;
    background-position: center;
    opacity: 0.8;
}}

.custom-markdown h1, p {{
    color: white;
}}
"""

with gr.Blocks(gr.themes.Monochrome(), css=css_code) as demo:
# with gr.Blocks(gr.themes.Default(primary_hue=gr.themes.colors.red, secondary_hue=gr.themes.colors.pink)) as demo:
    previous_tickers = gr.State(set())
    gr.Markdown("# 주식 가격 예측", elem_classes="custom-markdown")
    previous_tickers_display = gr.Radio(label="이전 조회된 티커", choices=[], interactive=True, visible=False)
    ticker_input = gr.Textbox(label="주식 티커 입력", placeholder=f"예: {", ".join(available_tickers[:5])} 등")
    warning_message = gr.Markdown(value="", visible=False)
    with gr.Row():
        submit_btn = gr.Button("조회")
        reset_btn = gr.Button("초기화")

    chart_output = gr.Image(label="주가 차트", visible=False)
    data_output = gr.Dataframe(label="일별 주가 데이터", visible=False)
    
    def update_visibility(ticker, previous_tickers):
        if not ticker:
            return gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="티커를 입력하세요.", visible=True, elem_classes="custom-markdown")
        
        ticker = ticker.upper()  # Convert ticker to uppercase
        if ticker.upper() not in available_tickers:
            return gr.update(visible=False), gr.update(visible=False), None, None, gr.update(choices=list(previous_tickers), visible=bool(previous_tickers)), gr.update(value="입력하신 종목의 가격 예측은 추후에 제공될 예정입니다..", visible=True)
        
        data, img = get_yfinance_stock_data(ticker)
        previous_tickers.add(ticker)
        return (
            gr.update(visible=True, label=f"일별 주가 데이터 [{ticker}]"),
            gr.update(visible=True),
            data,
            img,
            gr.update(choices=list(previous_tickers), visible=True),
            gr.update(visible=False)
        )
    
    def reset_fields():
        return "", gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)
    
    def update_ticker_input(selected_ticker):
        return selected_ticker, gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(value="", visible=False)
    
    def auto_submit_ticker(selected_ticker, previous_tickers):
        return update_visibility(selected_ticker, previous_tickers)
    
    submit_btn.click(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, data_output, chart_output, previous_tickers_display, warning_message])
    ticker_input.submit(update_visibility, inputs=[ticker_input, previous_tickers], outputs=[data_output, chart_output, data_output, chart_output, previous_tickers_display, warning_message])
    reset_btn.click(reset_fields, inputs=None, outputs=[ticker_input, data_output, chart_output, warning_message])
    previous_tickers_display.change(update_ticker_input, inputs=previous_tickers_display, outputs=[ticker_input, data_output, chart_output, warning_message])
    previous_tickers_display.change(auto_submit_ticker, inputs=[previous_tickers_display, previous_tickers], outputs=[data_output, chart_output, data_output, chart_output, previous_tickers_display, warning_message])

# Gradio 앱 실행
demo.launch()
# demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7924

To create a public link, set `share=True` in `launch()`.


[*********************100%***********************]  1 of 1 completed
